In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using LinearAlgebra
using CairoMakie
using YAML

# Background

__Problem statement__

Qubit 5 is driven to achieve an X gate. Qubit 6 has a $1 \leftrightarrow 2$ transition that is close to the $0 \leftrightarrow 1$ transition of qubit 5. As a result, the always-on coupling introduces leakage.


__Hamiltonian__
A pair of transmons is modeled using
\begin{align}
    H_\text{Transmon} &= \sum_{j=1,2} \omega_j a_j^\dag a_j + \frac{\alpha_j}{2} (a_j^\dag)^2 (a_j)^2 \\
    H_\text{Coupling} &= J (a_1^\dag a_2 + a_1 a_2^\dag) \\
    H_\text{Drive} &= \sum_{j=1,2} \varepsilon_j(t) e^{i \varphi_j(t)} a_j + \text{h.c.}
\end{align}

__Rotating frame__
Start with dynamics $i |\psi(t)\rangle = H(t) |\psi(t)\rangle$. For a state $|\tilde{\psi}(t)\rangle := U_R(t)^\dag |\psi(t)\rangle$ with $U_R(t) = \exp\{-i H_R t\}$, the rotating frame Hamiltonian is
\begin{equation}
    \tilde{H} = U^\dag(t) H(t) U(t) - H_R.
\end{equation}

Define the rotation axis of $U_R(t) = \exp\{-i H_R t\}$ in one of two ways:
\begin{align}
   H_R &= \omega_d (a_1^\dag a_1 + a_2^\dag a_2) \\
   H_R &= \omega_1 a_1^\dag a_1 + \omega_2 a_2^\dag a_2.
\end{align}
In either case,
\begin{equation}
    \tilde{H} = H_\text{Transmon} - H_\text{R}(t) +  U_R^\dag H_\text{Coupling} U_R(t) + U_R^\dag H_\text{Drive} U_R(t).
\end{equation}

__Choice__
For the AQT case, we are studying qubit 5 and 6. Perhaps we are only using control on qubit 5. In this case, we probably want to go to the frame where both qubits rotate at $\omega_5$. This frame will avoid any slowly varying time-dependence from frequency differences. Hence, $H_\text{Coupling}$ remains the same and $H_\text{Drive}$ is only driven by the control envelope, while $H_\text{Transmon}$ picks up an offset $(\omega_6 - \omega_5) a_6^\dag a_6$. Hiding here is the fact that we also make the RWA in the drive.

\begin{align}
    H_\text{Drift} &= (\omega_6 - \omega_5) a_6^\dag a_6 + \sum_{j=5,6}\frac{\alpha_j}{2} (a_j^\dag)^2 (a_j)^2 + J (a_5^\dag a_6 + a_5 a_6^\dag) \\
    H_\text{Drive} &= \sum_{j=5,6} \varepsilon^{I}_j(t) (a_j + a_j^\dag) + \varepsilon^{Q}_j(t) i (a_j - a_j^\dag).
\end{align}

# Compute

## Quantum utilities

A few utilities that aren't in quantum utils yet.

In [ ]:
"""
    embed_operator(subspace::AbstractVector{<:Integer}, dim::Int, operator::AbstractMatrix)

Embed the operator at the indices of subspace into a space of size dim.
"""
function embed_operator(subspace::AbstractVector{<:Integer}, dim::Integer, operator::AbstractMatrix)
    embedding = zeros(dim, dim)
    embedding[subspace, subspace] .= operator
    return embedding
end

"""
    embed_operators(levels::AbstractVector{<:Integer}, operators::AbstractVector)

Embed the provided operators into the larger Hilbert space defined by levels.
"""
function embed_operator(levels::AbstractVector{<:Integer}, operators::AbstractVector)
    subspace_levels = [size(H, 1) for H in operators]
    s = subspace_indices(subspace_levels, levels)
    return embed_operator(s, d, reduce(kron, operators))
end

embed_operator(level::Integer, operator::AbstractMatrix) = embed_operator([level], [operator])

"""
    truncate_operators(
        subspace_levels::AbstractVector{<:Integer},
        levels::AbstractVector{<:Integer},
        operator::AbstractMatrix)

Truncate the provided operator from the Hilbert space spanned be levels into the subspace_levels.
"""
function truncate_operator(
        subspace_levels::AbstractVector{<:Integer},
        levels::AbstractVector{<:Integer},
        operator::AbstractMatrix)
    s = subspace_indices(subspace_levels, levels)
    return operator[s, s]
end

"""
    truncate_operators(subspace_levels::AbstractVector{<:Integer}, operators::AbstractVector)

Truncate the list of operators into the subspace spanned by subspace_levels.
"""
truncate_operator(subspace_levels::AbstractVector{<:Integer}, operators::AbstractVector) =
    truncate_operator(subspace_levels, [size(H, 1) for H in operators], reduce(kron, operators))

truncate_operator(subspace_levels::Integer, operators::AbstractMatrix) =
    truncate_operator([subspace_levels], [operators])

In [ ]:
"""
operators_from_dict(keys::AbstractVector{<:Any}, operator_dictionary; I_key=:I)

    Replace the vector of keys using the operators from a dictionary.
"""
function operators_from_dict(keys::AbstractVector{<:Any}, operator_dictionary; I_key=:I)
    first_operator = first(values(operator_dictionary))
    I_default = Matrix{eltype(first_operator)}(I, size(first_operator))
    # Identity key is replaced by operator_dictionary, else default I.
    return replace(replace(keys, operator_dictionary...), I_key => I_default)
end

"""
operators_from_dict(key_string::String, operator_dictionary; I_key="I")

    Replace the string (each character is one key) with operators from a dictionary.
"""
operators_from_dict(key_string::String, operator_dictionary; I_key="I") = 
    operators_from_dict([string(c) for c ∈ key_string], operator_dictionary, I_key=I_key)

"""
kron_from_dict(keys, dict; kwargs...)

    Reduce the keys to a single operator by using the provided dictionary and the kronecker product.
"""
kron_from_dict(keys, dict; kwargs...) = reduce(kron, operators_from_dict(keys, dict; kwargs...))

## Problem

Set up problem parameters. 

In [ ]:
const Units = 1e9  # GHz, ns
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units

const n_qubits = 2
const n_levels = 3

In [ ]:
a_bound = 2 * π * 500 * MHz

n_steps = 101
t_f = 10 * ns
t_times = range(0, t_f, n_steps)
Δt = t_times[2] - t_times[1]

# Operators 
at = create(n_levels)
a = annihilate(n_levels)

H_operators = Dict(
        "X" => a + at,
        "Y" => -im * (a - at),
        "Z" => I - 2 * at * a,
        "N" => at * a,
        "A" => 1 / 2 * at * at * a * a,
);

In [ ]:
config = YAML.load_file("./config_X6Y3.yaml");

function get_AQT_transmon(config, index) 
    ω_ge = config["single_qubit"][index]["GE"]["freq"]
    α = (config["single_qubit"][index]["EF"]["freq"] - ω_ge)
    return 2 * π * ω_ge, 2 * π * α
end

ω_5, α_5 = get_AQT_transmon(config, 5) ./ Units
ω_6, α_6 = get_AQT_transmon(config, 6) ./ Units
ω_d = ω_5

J_56 = config["two_qubit"]["(5, 6)"]["CZ"]["freq"] / Units

In [ ]:
probs = Dict()

### Isolated Qubit 5

Equivalent of drag pulse from optimal control, only difference now is transmon instead of qubit.

In [ ]:
H_drift = (
    (ω_5 - ω_d) .* kron_from_dict("N", H_operators)
    + α_5 .* kron_from_dict("A", H_operators)
)

H_controls = [
    kron_from_dict("X", H_operators),
    kron_from_dict("Y", H_operators)
]

# Currently, the operator must be embedded into a matrix of zeros at the correct indices.
# This will have to change in future versions.
subspace = [1,2]
U_goal = embed_operator(subspace, n_levels, GATES[:X])

probs["5"] = UnitarySmoothPulseProblem(
    H_drift,
    H_controls,
    U_goal,
    n_steps,
    Δt;
    subspace=subspace,
    geodesic=false,
    free_time=true,
    timesteps_all_equal=false,
    a_bound=a_bound / 10,
    dda_bound=1.0,
    hessian_approximation=true,
    pade_order=4,
    R_a=1e-3,
    R_da=1e-3,
    R_dda=1e-3,
)


In [ ]:
solve!(probs["5"]; max_iter=200)

unitary_fidelity(probs["5"]; subspace=subspace)

In [ ]:
plot(probs["5"].trajectory)

### Qubit 5, Ignoring Qubit 6

Use the full Hamiltonian, but only focus on getting a good gate for Qubit 5.

In [ ]:
H_drift = (
    (ω_5 - ω_d) .* kron_from_dict("NI", H_operators)
    + (ω_6 - ω_d) .* kron_from_dict("IN", H_operators)
    + α_5 .* kron_from_dict("AI", H_operators)
    + α_6 .* kron_from_dict("IA", H_operators)
    + J_56 .* (a ⊗ a' + a' ⊗ a)
)

H_controls = Matrix{ComplexF64}[
    kron_from_dict("XI", H_operators),
    kron_from_dict("YI", H_operators),
    kron_from_dict("IX", H_operators),
    kron_from_dict("IY", H_operators)
]

# Currently, the operator must be embedded into a matrix of zeros at the correct indices.
# This will have to change in future versions.
subspace = [1,2]
U_goal = embed_operator(subspace, n_levels^n_qubits, GATES[:X])

probs["56"] = UnitarySmoothPulseProblem(
    H_drift,
    H_controls,
    U_goal,
    n_steps,
    Δt;
    subspace=subspace,
    geodesic=false,
    free_time=true,
    timesteps_all_equal=false,
    a_bound=a_bound,
    dda_bound=10.,
    hessian_approximation=true,
    pade_order=4,
    R_a=1e-3,
    R_da=1e-3,
    R_dda=1e-1,
)

In [ ]:
solve!(probs["56"]; max_iter=100)

unitary_fidelity(probs["56"]; subspace=subspace)

In [ ]:
plot(probs["56"].trajectory)

In [ ]:
# Extend function
import QuantumCollocation: unitary_rollout

function unitary_rollout(prob::QuantumControlProblem; kwargs...)
    return unitary_rollout(prob.trajectory, prob.system; kwargs...)
end

In [ ]:
U_final = iso_vec_to_operator(unitary_rollout(probs["56"])[:, end])

# reproduce fidelity from before
abs(tr(U_final[1:2, 1:2]'GATES[:X])) / 2

In [ ]:
# We want the last two indices for the gate on the other qubit
subspace_indices([3, 3]) |> println

# Some gate was applied
U_final[4:5, 4:5]

### Qubit 5 & 6 together

Previously, we ignore the impact on qubit 6. Now enforce as identity.

In [ ]:
H_drift = (
    (ω_5 - ω_d) .* kron_from_dict("NI", H_operators)
    + (ω_6 - ω_d) .* kron_from_dict("IN", H_operators)
    + α_5 .* kron_from_dict("AI", H_operators)
    + α_6 .* kron_from_dict("IA", H_operators)
    + J_56 .* (a ⊗ a' + a' ⊗ a)
)

H_controls = Matrix{ComplexF64}[
    kron_from_dict("XI", H_operators),
    kron_from_dict("YI", H_operators),
    kron_from_dict("IX", H_operators),
    kron_from_dict("IY", H_operators)
]

# Currently, the operator must be embedded into a matrix of zeros at the correct indices.
# This will have to change in future versions.
subspace = subspace_indices([3, 3])
U_goal = embed_operator(subspace, n_levels^n_qubits, GATES[:X] ⊗ GATES[:I])

probs["XI"] = UnitarySmoothPulseProblem(
    H_drift,
    H_controls,
    U_goal,
    n_steps,
    Δt;
    subspace=subspace,
    geodesic=false,
    free_time=true,
    timesteps_all_equal=false,
    a_bound=a_bound,
    dda_bound=10.,
    hessian_approximation=true,
    pade_order=4,
    R_a=1e-3,
    R_da=1e-3,
    R_dda=1e-3,
)

In [ ]:
solve!(probs["XI"]; max_iter=200)

unitary_fidelity(probs["XI"]; subspace=subspace)

__TODO__ Simulate the effect of this crosstalk on the standard X gate. Can we observe that the leakage is is happening? What is the impact of these alternative pulses?

In [ ]:
plot(probs["XI"].trajectory)

In [ ]:
U_final = iso_vec_to_operator(unitary_rollout(probs["XI"])[:, end]);

In [ ]:
abs(tr(U_final[subspace, subspace]' * (GATES[:X] ⊗ GATES[:I]))) / 4

In [ ]:
round.(abs.(U_final[subspace, subspace]); digits=1)

In [ ]:
GATES[:X] ⊗ GATES[:I]

### Robust qubit 5

The idea is now to find a gate for qubit 5 using pulses on qubit 5 only.